<a href="https://colab.research.google.com/github/PratikshaSN/Python/blob/main/TMWithUserAuthentication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cryptocode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.4 MB/s eta 0:00:00


In [9]:
# Course-End Project - Task Manager with User Authentication

import os
import getpass
import cryptocode
import json

from datetime import datetime
import time

def mencrypt(mstring):
    passkey = '\a'
    cipher = cryptocode.encrypt(mstring,passkey)
    return cipher

def mdecrpyt(mcipher):
    string=cryptocode.decrypt(mcipher,'\a')
    return string

#Check username in the list
def check_username(user_name):
    for user in users:
        if mdecrpyt(user['name'])==user_name:
            return True
    else:
        return False

#Check user registration
def check_user_registration():
    logged_in=False
    global current_user

    print("\n## Registration ##\n")
    are_you=input("Are you registered user? (y/n): ")

    if are_you.lower()=='y':
        user_name=input("\nEnter username: ")
        for user in users:
            if mdecrpyt(user['name'])==user_name:
                user_pass = getpass.getpass('Password:')
                if mdecrpyt(user['pass'])==user_pass:
                    print(f"\nUser: {user_name} successfully logged in..\n")
                    current_user=user
                    logged_in=True
                    ud=int(time.time())
                    user['updatetm']=ud
                else:
                    print("\nInvalid password. Please try again..\n")
    else:
        print("\nUser Registration\n ")
        user={}
        user_name=input("\nEnter username: ")
        if user_name=="":
            print("\nNo username entered exiting..\n")
            exit(1)

        while check_username(user_name):
            print("Username already exists.. enter different username")
            user_name=input("Enter username: ")
            if user_name=="":
                print("No username entered. Exiting....")
                exit(1)

        user_email_id=input("Enter email id: ")
        user_pass1=getpass.getpass(prompt='Enter password: ')
        user_pass2=getpass.getpass(prompt='Enter password again: ')

        if user_pass1==user_pass2:
            user['name']=mencrypt(user_name)
            user['pass']=mencrypt(user_pass1)
            user['email']=user_email_id
            user['createtm']=int(time.time())
            user['updatetm']=int(time.time())
            users.append(user)
            register_file = 'register.fil'
            with open(register_file, "a+") as f:
                f.write("{}:{}:{}:{}:{}\n".format(user['email'],user['name'],user['pass'],user['createtm'],user['updatetm']))
            print(f"\nUser:{user_name} added successfully and logged in.\n")
            current_user=user
            logged_in=True
    return logged_in

# Store task details file
def store_task_details_file():
    task_file = "usertasks.json"
    with open(task_file, 'w+') as file:
            json.dump(usertasks, file, ensure_ascii=False)


def load_task_details_file():
    current_email=current_user['email']
    current_ctm=current_user['createtm']
    global usertasks
    task_file = "usertasks.json"
    if os.path.exists(task_file):
        mode='r'
    else:
        mode='w+'

    with open(task_file, mode) as file:
        if mode=='r':
            usertasks=json.load(file)
        else:
            user_json='{"usertasks":[]}'
            json.dump(user_json, file, ensure_ascii=False)
            usertasks=json.loads(user_json)

def load_registration():
    register_file = 'register.fil'
    if os.path.exists(register_file):
        mode='r'
    else:
        return

    with open(register_file, mode) as f:
        reg_readline=f.read()

    for u in reg_readline.splitlines():
            user={}
            user['email'],user['name'],user['pass'],user['createtm'],user['updatetm']=u.strip().split(':')
            users.append(user)


def view_tasks_details(user_mail, usertasks):
    tasks=[]
    for user in usertasks['usertasks']:
        if user['mail'] == user_mail:
            tasks=user['tasks']
    if tasks:
        print("\nTaskID\tStatus  \tDescription\n")
        for tasklist in tasks:
            for k,v in tasklist.items():
                print(f"{v}  \t", end="")
            print()
        print("\n")
        v=input("\nView the Task List..\nPress ENTER key to continue\n")
        return True

    else:
        print(f"\nNo tasks found for {user_mail}\n")
        return False

    v=input("")

def add_tasks_details(user_mail, usertasks):
    v_status=view_tasks_details(user_mail, usertasks)
    a_taskitem={}
    a_taskId=input("Enter task id to Add: ")
    a_taskDescription=input("Enter task description: ")
    a_taskitem["taskid"]=a_taskId
    a_taskitem["status"]="pending"
    a_taskitem["detail"]=a_taskDescription
    for usertask in usertasks['usertasks']:
        if usertask['mail'] == user_mail:
            usertask['tasks'].append(a_taskitem)
            break
    else:
        usertasks['usertasks'].append({"mail": user_mail, "tasks": [a_taskitem]})

    store_task_details_file()
    v=input(f"\nAdded TaskID: {a_taskId}\nPress ENTER key to continue\n")
    return True


def delete_task(user_mail, usertasks):
    v_status=view_tasks_details(user_mail, usertasks)
    delete_task=False
    d_taskId=input("Enter task id to delete: ")
    for usertask in usertasks['usertasks']:
        if usertask['mail'] == user_mail:
            usertask['tasks'] = [task for task in usertask['tasks'] if task['taskid'] != d_taskId]
            delete_task=True
            break
    if delete_task:
        store_task_details_file()
        v=input(f"\nDeleted TaskID: {d_taskId}\nPress ENTER key to continue\n")
        return True

def modify_task(user_mail, usertasks):
    v_status=view_tasks_details(user_mail, usertasks)
    a_taskitem={}
    updated_task=False
    a_taskId=input("Enter task id to mark completed: ")
    updated_task={"status": "completed"}
    for usertask in usertasks['usertasks']:
        if usertask['mail'] == user_mail:
            for task in usertask['tasks']:
                if task['taskid'] == a_taskId:
                    task['status']="completed"
                    updated_task=True
                    break
    if updated_task:
        store_task_details_file()
        v=input(f"Updated TaskID: {a_taskId} as completed")
        print("Press ENTER key to continue")
        return True

## main  task manager with user authentication program

users=[]
current_user={}
usertasks={}

load_registration()
if not check_user_registration():
    print("\nNo login found exiting..\n")
    exit(1)

load_task_details_file()

while True:

    print(f"""

    ##  Task Management System - {current_user['email']} ##

    1.. Add a Task
    2.. View Tasks
    3.. Mark a Task as Completed
    4.. Delete a Task

    5.. Logout

        """)

    choice=int(input("Enter your choice: "))

    if choice==1:
        print(f"\n## Add a Task - {current_user['email']} ##\n")
        add_tasks_details(current_user['email'], usertasks)
    elif choice==2:
        print(f"\n## View Tasks - {current_user['email']} ##\n")
        v_status=view_tasks_details(current_user['email'], usertasks)
    elif choice==3:
        print(f"\n## Mark a Task as Complete - {current_user['email']} ###\n")
        modify_task(current_user['email'], usertasks)
    elif choice==4:
        print(f"\n## Delete a Task - {current_user['email']} ##\n")
        delete_task(current_user['email'], usertasks)
    elif choice==5:
        print(f"\n## Logout - {current_user['email']} ##\n")
        break
    else:
        print("\nInvalid choice. Try again..\n")


# Done.


## Registration ##

Are you registered user? (y/n): n

User Registration
 

Enter username: Kunal1
Enter email id: Kunal1@gmail.com
Enter password: ··········
Enter password again: ··········

User:Kunal1 added successfully and logged in.



    ##  Task Management System - Kunal1@gmail.com ##
    
    1.. Add a Task
    2.. View Tasks
    3.. Mark a Task as Completed
    4.. Delete a Task

    5.. Logout

        
Enter your choice: 1

## Add a Task - Kunal1@gmail.com ##


No tasks found for Kunal1@gmail.com

Enter task id to Add: 123
Enter task description: Lunch

Added TaskID: 123
Press ENTER key to continue



    ##  Task Management System - Kunal1@gmail.com ##
    
    1.. Add a Task
    2.. View Tasks
    3.. Mark a Task as Completed
    4.. Delete a Task

    5.. Logout

        
Enter your choice: 2

## View Tasks - Kunal1@gmail.com ##


TaskID	Status  	Description

123  	pending  	Lunch  	



View the Task List..
Press ENTER key to continue



    ##  Task Management System 